In [1]:
import dataclasses

import jax
import jax.numpy as jnp


### Jax Pallas Experimental Flash Attention

Github: https://github.com/jax-ml/jax/blob/922935a916dbcf599226f1ce3081feb6481328c3/jax/experimental/pallas/ops/gpu/attention.py

In [2]:
rng = jax.random.PRNGKey(0)

max_seq_len = 64
B, T, nH, C = 32, 15, 8, 128

q = jax.random.normal(rng, (B, T, nH, C), dtype=jnp.bfloat16)
k = jax.random.normal(rng, (B, T, nH, C), dtype=jnp.bfloat16)
v = jax.random.normal(rng, (B, T, nH, C), dtype=jnp.bfloat16)

In [ ]:
from jax.experimental.pallas.ops.gpu.attention import mha

@dataclasses.dataclass(frozen=True, slots=True)
class BlockSizes:
  block_q: int
  block_k: int

pad_width = ((0, 0),  # no padding on the first dimension
             (0, max_seq_len-T),
             (0, 0),
             (0, 0))  # pad two zeros on the right side of the second dimension

q = jnp.pad(q, pad_width, mode='constant', constant_values=0)
k = jnp.pad(k, pad_width, mode='constant', constant_values=0)
v = jnp.pad(v, pad_width, mode='constant', constant_values=0)

block_sizes = BlockSizes(block_q=T, block_k=T)

#y = mha(q, k, v, block_sizes=block_sizes, segment_ids=None, causal=True)
%timeit mha(q, k, v, segment_ids=None, causal=True)

### FlashAttention Jax

Github: https://github.com/nshepperd/flash_attn_jax

In [ ]:
!export CUDA_HOME=/usr/
!uv add flash-attn-jax

In [ ]:
from flash_attn_jax import flash_mha

# flash_mha : [n, l, h, d] x [n, lk, hk, d] x [n, lk, hk, d] -> [n, l, h, d]
%timeit flash_mha(q,k,v,softmax_scale=None, is_causal=True, window_size=(-1,-1))

### Flash Attention - Jax
Github: https://github.com/lucidrains/flash-attention-jax

In [ ]:
!pip install flash-attention-jax

In [ ]:
from flash_attention_jax import causal_flash_attention

rng_key = jax.random.PRNGKey(42)

%timeit causal_flash_attention(q, k, v)

### Kvax

Github: https://github.com/nebius/kvax

In [ ]:
!pip install kvax

### Nvidia TransformerEngine

Github: https://github.com/NVIDIA/TransformerEngine

In [ ]:
!pip install git+https://github.com/NVIDIA/TransformerEngine.git@stable

